# Mixture of Gaussian Experiments

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility func

In [2]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [3]:
# plot images
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    # make directory if there is not
    path = "generated_figures"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 60
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((96, 96, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

In [4]:
def fc(x, output_shape, name):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    initializer = tf.random_normal_initializer(0., 0.001)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        bias = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
        x = tf.nn.bias_add(x, bias)

    return x

In [5]:
import math
def gaussian_mixture_circle(batch_size, num_cluster=8, scale=1, std=1):
    rand_indices = np.random.randint(0, num_cluster, size=batch_size)
    base_angle = math.pi * 2 / num_cluster
    angle = rand_indices * base_angle - math.pi / 2
    mean = np.zeros((batch_size, 2), dtype=np.float32)
    mean[:, 0] = np.cos(angle) * scale
    mean[:, 1] = np.sin(angle) * scale
    return np.random.normal(mean, std**2, (batch_size, 2)).astype(np.float32)

# model

In [6]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.z_dim = 100
        self.h_dim = 128
        self.o_dim = 2
        self.initializer = tf.contrib.layers.xavier_initializer()

    def __call__(self, x, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.z_dim, self.h_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.h_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias
                x = tf.nn.relu(x)

            with tf.variable_scope("fc2"):
                weight = tf.get_variable("W", shape=[self.h_dim, self.o_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.o_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return x

In [7]:
class Discriminator:
    def __init__(self, batch_size=32):
        self.reuse = False
#        self.initializer = tf.truncated_normal(stddev=0.8)
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 2
        self.o_dim = 1
        
    def __call__(self, x, training=False, name=''):
        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = x * 0.25

            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.X_dim, self.o_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.o_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias
                x = tf.nn.relu(x)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return x

In [8]:
class GAN:
    def __init__(self, batch_size=64):
        self.batch_size = batch_size
        self.g = Generator()
        self.d = Discriminator()
        self.z_dim = 100
        self.batch_size = batch_size

    def loss(self, y, y_gen):
        d_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=tf.ones_like(y)))
        g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_gen, labels=tf.zeros_like(y_gen)))   
        return g_loss, d_loss

    def train(self, batch_size=64, epochs=500000):
        z = tf.placeholder(tf.float32, shape=[None, self.z_dim])
        x = tf.placeholder(tf.float32, shape=[None, 2])
        x_gen = self.g(z)
        y = self.d(x)
        y_gen = self.d(x_gen)
        g_loss, d_loss = self.loss(y, y_gen)
        
        t_vars = tf.global_variables()
        g_vars = [var for var in t_vars if "g" in var.name]
        d_vars = [var for var in t_vars if "d" in var.name]

        opt_d = tf.train.AdamOptimizer(1e-4, beta1=0.5).minimize(d_loss, var_list=d_vars)
        opt_g = tf.train.AdamOptimizer(1e-4, beta1=0.5).minimize(g_loss, var_list=g_vars)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            # 潜在変数と生成データ
            z_gen = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])

            # 訓練データを生成
            X_tr = gaussian_mixture_circle(self.batch_size, num_cluster=8, scale=2, std=0.02)
            print(X_tr.shape)

            _, dis_loss = self.sess.run([opt_d, d_loss], feed_dict={z: z_gen, x: X_tr})
            _, gen_loss = self.sess.run([opt_g, g_loss], feed_dict={z: z_gen, x: X_tr})

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [9]:
if __name__ == "__main__":
    gan = GAN()
    gan.train()

(64, 2)


InternalError: Blas SGEMM launch failed : a.shape=(64, 2), b.shape=(2, 1), m=64, n=1, k=2
	 [[Node: d/d/fc1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](d/d/mul, d/fc1/W/read)]]

Caused by op 'd/d/fc1/MatMul', defined at:
  File "/home/ujitoko/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ujitoko/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ujitoko/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/ujitoko/anaconda3/lib/python3.6/site-packages/zmq/eventlo

In [ ]:
import numpy as np

X_train = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
X_train = X_train/255
X_dim = 96*96*3
z_dim = 100
batch_size = 32
epochs = 500000
display_epoch = 100
param_save_epoch = 10000
loss = {"dis_loss":[], "gen_loss":[]}

# main func

In [ ]:
p_noise = tf.placeholder(tf.float32, [None, z_dim])
noise_check = np.random.uniform(-1, 1, size=[60, z_dim]).astype(np.float32)

Xr = tf.placeholder(tf.float32, [None, X_dim])
Z1 = tf.placeholder(tf.float32, [None, z_dim])
Z2 = tf.placeholder(tf.float32, [None, z_dim])

gan = GAN()

losses = gan.loss(Xr=Xr, z1=Z1, z2=Z2)
d_train_op = gan.d_train(losses)
g_train_op = gan.g_train(losses)

saver = tf.train.Saver()
#%debug
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        if epoch % display_epoch == 0:
            imgs_gen = gan.sample_images(inputs=noise_check).eval()
            print("saving images")
            save_imgs(imgs_gen, epoch=epoch)

        for _ in range(5):
            # 訓練データを抜粋
            rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
            X_mb = X_train[rand_index, :].astype(np.float32)
            X_mb = X_mb.reshape([-1, X_dim])

            z1 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
            z2 = np.random.uniform(-1, 1, size=[batch_size, z_dim])

            _, d_loss_value = sess.run([d_train_op, losses[gan.d]], feed_dict={Xr: X_mb, Z1:z1, Z2:z2})


        # 訓練データを抜粋
        rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
        X_mb = X_train[rand_index, :].astype(np.float32)
        X_mb = X_mb.reshape([-1, X_dim])
        z1 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
        z2 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
        _, g_loss_value, = sess.run([g_train_op, losses[gan.g]], feed_dict={Xr: X_mb, Z1:z1, Z2:z2})

        # 結果をappend
        loss["dis_loss"].append(d_loss_value)
        loss["gen_loss"].append(g_loss_value)
        print("epoch:" + str(epoch))
        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            save_metrics(loss, epoch)

        if epoch % param_save_epoch == 0:
            path = "model"
            if not os.path.isdir(path):
                os.makedirs(path)

            saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")